# Рекомендация тарифов

**Цель проекта:**

На основании данных о поведении клиентов оператора мобильной связи "Мегалайн", построить модель для рекомендации подходящего актуального тарифного плана ("Смарт" или "Ультра").

Модель будет сформирована на основании данных о поведении пользователей, которые уже перешли на эти тарифы.

**Описание процесса моделирования:**

Формирование модели будет выполнено в 5 этапов:

- Обзор датасета (данные в датасете уже предобработаны);
- Разделение датасета на выборки (тренировочная, валидационная и тестовая);
- Исследование различных моделей рекомендации тарифа (модель для задачи классификации);
- Проверка моделей на тестовой выборке;
- Оценка выбранной модели на адекватность.

## Обзор датасета

Для формирвоание модели классификации подключаем методы библиотеки sklearn: дерево решений, случайный лес и логическая регрессия.

Для разделения датасета на выборки подключаем метод train_test_split.

Для оценки качества модели - метод accuracy_score.

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [ ]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [ ]:
df.tail()

,calls,minutes,messages,mb_used,is_ultra
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


**Вывод по обзору датасета:**

Датасет содержит информацию о звонках, минутах, сообщениях, траффике интернета и признак тарифа "Ультра". Учитывая, что всего тарифа два, то строки, соответствующие значению 0 в графе "is_ultra" - означают, что такие пользователи выбрали тариф "Смарт".
Пропусков в данных нет, тип данных соотвествует значаниям в соответсвующих графах. По среднему значению в графе "is_ultra" можно сказать, что чуть более 30% пользователей выбрали тариф "Ультра", а большинство использует тариф "Смарт".

In [ ]:
df.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


В датасете наблюдается наличие двух столбцов данных ('calls' и 'minutes'), между которыми присутствует очень высокая, близкая к единице (98.2%) корреляция. Это означает наличие прямой связи между количеством звонков и потраченных минутах, как следствие, для обучения моделей, один из этих признаков будет излишним, поэтому при формировании датафрейма признаков, исключим столбец 'calls', исключение которого, с учетом выявленной корреляции не должно повлиять на качество работы моделей.

## Разделение датасета на выборки

Формируем датафрейм признаков и датафрейм целевого признака.

In [ ]:
features = df.drop(['is_ultra', 'calls'], axis=1)
target = df['is_ultra']

Так как отдельной тестовой выборки нет - сформируем тестовую, валидационную и тренировочную выборки из датасета. В таком случае датасет разбивают пропорционально: 3:1:1, где 3 - коэффициент тренировочной выборки.

Формирование выборок проведем последовательно в 2 этапа:

- Сначала выделим тренировочную выборку - 60% от датасета
- Потом разделим поровну оставшиеся 40%, получив тем самым валидационную и тестовую выборки.

In [ ]:
features_train, features_temp, target_train, target_temp = \
          train_test_split(features, target, test_size=0.4, random_state=12345)

features_test, features_valid, target_test, target_valid = \
         train_test_split(features_temp, target_temp, test_size=0.5, random_state=12345)

In [ ]:
print(features_train.shape, features_test.shape, features_valid.shape)
print(target_train.shape, target_test.shape, target_valid.shape)

(1928, 3) (643, 3) (643, 3)
(1928,) (643,) (643,)


Проверена размерность полученных выборок. Размерность соответствует требуемой пропорции 3:1:1.

## Исследование различных моделей рекомендации тарифа

Проедем последовательное исследование 3 моделей: Решающее дерево, Случайный лес и логическая регрессия.

Модель **Решающее дерево**

In [ ]:
best_model = None
best_result = 0
best_depth = 0

for i in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=i)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)

    if result > best_result:
        best_model = model
        best_result = result
        best_depth = i

print("Качество наилучшей модели на валидационной выборке:", best_result.round(4), "Глубина дерева:", best_depth)

Качество наилучшей модели на валидационной выборке: 0.7963 Глубина дерева: 7


Так как в цикле просматриваем варианты глубины деревьев от 1 до 10, а наилучшее результаты достигаются при глубине 7, дальнейшее изменение (поиск) параметра максимальной глубины дерева нецелесообразно.

Модель **Случайный лес**

In [ ]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0

for est in range(10, 41, 5):
    for i in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=i)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid)

        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = i

print("Качество наилучшей модели на валидационной выборке:", best_result.round(4), \
      "Количество деревьев:", best_est, "Максимальная глубина:", best_depth)

Качество наилучшей модели на валидационной выборке: 0.8087 Количество деревьев: 10 Максимальная глубина: 8


Так как в цикле просматриваем варианты глубины деревьев от 1 до 10 и количество деревьев от 10 до 40, а наилучшее результаты достигаются при глубине 8 и количестве деревьев 10, дальнейшее изменение (поиск) параметра максимальной глубины и количества деревьев нецелесообразно.

Модель **Логическая регрессия**

In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)

print("Качество модели на валидационной выборке:", result.round(4))

Качество модели на валидационной выборке: 0.6983


**Вывод по разделу**

Рассмотрены три модели: Решающее дерево, Случайный лес и Логическая регрессия.

Для моделей Решающее дерево и Случайный лес произведен поиск лучших гиперпараметров (глубина дерева, количество деревьев) по критерию точности моделей. Определены гиперпараметры, дающие максимальную точность на валидационной выборке.

По результатам расчета точности моделей лучшей признана модель **Случайный лес** с гиперпараметрами: глубина деревьев - 8, количество деревьев - 10. Такая модель показала точность на валидационной выборке = **80.9%**

## Проверка моделей на тестовой выборке

Последовательно проверим модели с найденными лучшими гиперпараметрами на тестовой выборке.

Проверим все модели, начиная с лучшей по валидационной выборке, т.к. результаты на валидационной выборке не всегда совпадают с результатами на тестовой.

In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=10, max_depth=8)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_test)
print("Точность модели Случайный лес на тестовой выборке:", result.round(4))

Точность модели Случайный лес на тестовой выборке: 0.7838


In [ ]:
model = DecisionTreeClassifier(random_state=12345, max_depth=7)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_test)
print("Точность модели Дерево решений на тестовой выборке:", result.round(4))

Точность модели Дерево решений на тестовой выборке: 0.7714


In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_test)
print("Точность модели Логическая регрессия на тестовой выборке:", result.round(4))

Точность модели Логическая регрессия на тестовой выборке: 0.7076


**Вывод по разделу:**

Модель, показавшая лучший результат точности на валидационной выборке, продемонстрировала также лучший результат и на тестовой.

Итого. Выбираем лучшую модель **Случайный лес** с гиперпараметрами: глубина дерева - 8, количество деревьев - 10.

Точность у данной модели на тестовой выборке составила **78.4%**, что выше заданной минимальной точности равной 75%, таким образом, данная модель удовлетворяет требованию заказчика.

## Проверка модели на адекватность

Проверку мождели на адекватность проведем сравнив результаты работы модели на всем датасете с данными датасета - прогнозируемое и реальное (фактическое) значение в графе "is_ultra".

In [ ]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

В датасете всего 985 пользователей с тарифом "Ультра". Всего пользователей в выборке 3 214. Таким образом, вероятность случайно угадать пользователя с тарифом "Ультра" составляет 30.6%
Точность модели и на валидационной и на тестовой выборках гораздо выше и составляет порядка 79%.

In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=10, max_depth=8)
model.fit(features_train, target_train)
predictions = model.predict(features)
df['is_ultra_predict']=predictions

In [ ]:
df['is_ultra_predict'].value_counts()

0    2580
1     634
Name: is_ultra_predict, dtype: int64

По результатам работы модели предсказано 634 пользователей с тарифом "Ультра" из 985. Т.е. точность определеления для тарифа "Ультра" составила 64.4% для всего датасета, что также существенно выше вероятности случайно угадать данный тариф.

In [ ]:
df.head(20)

,calls,minutes,messages,mb_used,is_ultra,is_ultra_predict
0,40.0,311.90,83.0,19915.42,0,0
1,85.0,516.75,56.0,22696.96,0,0
2,77.0,467.66,86.0,21060.45,0,0
3,106.0,745.53,81.0,8437.39,1,0
4,66.0,418.74,1.0,14502.75,0,0
5,58.0,344.56,21.0,15823.37,0,0
6,57.0,431.64,20.0,3738.90,1,0
7,15.0,132.40,6.0,21911.60,0,0
8,7.0,43.39,3.0,2538.67,1,0
9,90.0,665.41,38.0,17358.61,0,0


При просмотре части данных датасета видно, что модель не предстказывает тариф "Ультра" для тарифов "Смарт" (там где is_ultra равно 0, модель также предсказывает 0). Но для часть пользователей с тарифом "Ультра" модель предсказывает тариф "Смарт". Таким образом, наблюдается односторонняя ошибка модели, объяснимая, на мой взгляд, как объемом данных самого датасета, так и итоговой точностью выбранной модели.

**Вывод по разделу:**

Выбранная модель подтверждает свою адекватность при дальшейшем ее исследовании.

**ОБЩИЙ ВЫВОД:**
На датасете поведения пользователей, выбравших актуальные тарифы компании "Мегалайн" в объеме 3 214 строк была создана модель рекомендации тарифов, в основе которой лежит модель **Случайный лес**. Данная модель показала высокую точность как на валидационной (80.9%), так и на тестовой (**78.4%**) выборках, а также прошла проверку на адекватность.
В связи с тем, что минимальная точность модели была определена заказчиком в 75%, данная модель может быть использована компанией "Мегалайн" для рекомендации актуальных тарифов для пользователей.